### 분석전 작업

In [135]:
# 라이브러리
import pandas as pd
import numpy as np

In [136]:
## 데이터 불러오기
age_1 = pd.read_csv('C:/sas_view/data/real_data1.csv',index_col=0)
age_2 = pd.read_csv('C:/sas_view/data/real_data2.csv',index_col=0)

C:\Users\dkq50\AppData\Local\Temp\ipykernel_11176\2898011333.py:2: DtypeWarning: Columns (31,32,202,265,277,288,291,331,372,399,411,437,443,464,475,482,510,518) have mixed types. Specify dtype option on import or set low_memory=False.
  age_1 = pd.read_csv('C:/sas_view/data/real_data1.csv',index_col=0)


### 상관계수를 포함한 데이터 프레임 생성

1. 질병, 변수, 상관계수 등 필요 데이터 추출 작업 실시

In [21]:
dis = [] # 질병 리스트
var = [] # 변수 리스트
Cor = [] # 상관계수 리스트

for i in range(1,age_2.shape[1]): # shape : 행열의 개수 추출 / shape[1] : 열의 수
    for j in range(1,age_1.shape[1]): 
        try:
            corr = age_2.iloc[:,i].corr(age_1.iloc[:,j])
            if ((corr >= .4) | (corr <= -.4)) & (corr !=1.000): # 상관계수가 0.4 이상인 데이터 추출
                dis.append(age_2.columns[i])
                var.append(age_1.columns[j])
                Cor.append(round(corr,3))
                
        except:
            pass

c:\Users\dkq50\anaconda3\envs\conda38\lib\site-packages\numpy\lib\function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\dkq50\anaconda3\envs\conda38\lib\site-packages\numpy\lib\function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


2. 추출된 리스트 활용하여 cor_df라는 상관계수를 포함한 데이터 프레임 생성

In [22]:
## 상관계수 df 만들기
cor_df = pd.DataFrame({'dis':dis,  # {'변수명' : 데이터를 가지고 있는 리스트}
                       'var':var,
                       'Cor':Cor})
cor_df

,dis,var,Cor
0,DI1_PR,DI1_PT,0.820
1,DI1_PR,DI1_2,-0.773
2,DI1_PR,DC12_AG,-0.693
3,DI1_PR,HE_HP,0.704
4,DI1_PR,DC12_PR,-1.000
...,...,...,...
512,DK4_PR,DC11_PR,-0.577
513,DK4_PR,DC11_PT,-0.577
514,DK4_PR,DK4_PT,0.734
515,DK4_PR,BO1_2,-0.413


3. cor_df을 쉽게 해석하기 위해 미리 만들어놓은 변수사전(same_code_2)을 활용하여 변수설명 추가 작업
* same_code_2 : 7차8차의 변수들을 모두 모아 놓은 사전 (변수명과 변수설명들이 동일)

In [138]:
####
same = pd.read_csv('C:/sas_view/code/same_code_2.csv',index_col=0)

In [139]:
for i in range(0,len(same)):
    same.iloc[i][0] = same.iloc[i][0].upper() ## 변수의 대소문자 차이를 없애기 위해 대문자로 모두 변경

same

,변수명,변수설명7,변수설명8
0,MOD_D,최종 DB 수정일,최종 DB 수정일
1,ID,개인 아이디,개인 아이디
2,ID_FAM,가구 아이디,가구 아이디
3,YEAR,조사연도,조사연도
4,REGION,17개 시도,17개 시도
...,...,...,...
747,T_NQ_OCP,작업장 내 소음노출 경험,직업적 소음노출
748,HE_DM,당뇨병 유병여부(19세이상),당뇨병 유병여부(19세이상)
749,TOTAL_SLP_WK,(만12세이상) 주중 하루 평균 수면시간,(만12세이상) 주중(또는 일하는 날) 하루 평균 수면 시간
750,TOTAL_SLP_WD,(만12세이상) 주말 하루 평균 수면시간,"(만12세이상) 주말(또는 일하지 않는 날, 일하지 않는 전날) 하루 평균 수면 시간"


4. cor_df 데이터프레임 구성 확인

In [140]:
dis_s = set(cor_df.iloc[:,0])
dis_s = list(dis_s)
len(dis_s) # 질병에 해당하는 변수의 수

28

In [141]:
var_s = set(cor_df.iloc[:,1])
var_s = list(var_s)
len(var_s) # 질병에 영향을 미치는 변수의 수

212

In [142]:
## 질병과 그에 영향을 미치는 변수를 리스트에 담아주는 작업 실시
dis_list = []
var_list = []
for  i in range(0,len(same)):
    if same.loc[i][0] in dis_s:
        dis_list.append(i)
    elif same.loc[i][0] in var_s:
        var_list.append(i)

print(len(dis_list),len(var_list))

28 212


5. 변수사전에서 활용할 데이터 분할작업

In [143]:
dis_df = same.iloc[dis_list] ## 질병에 해당하는 사전
var_df = same.iloc[var_list] ## 기본변수에 해당하는 사전

6. merge 작업전 변수명 맞추기 & merge

In [144]:
# 상관계수 df에 변수설명을 merge하기위해 변수사전에 있는 변수명을 cor_df에 맞추어 변경하는 작업 실시
dis_df.rename(columns={'변수명':'dis','변수설명7':'dis_설명'},inplace=True) 
var_df.rename(columns={'변수명':'var','변수설명7':'var_설명'},inplace=True)

C:\Users\dkq50\AppData\Local\Temp\ipykernel_11176\1408335952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_df.rename(columns={'변수명':'dis','변수설명7':'dis_설명'},inplace=True)
C:\Users\dkq50\AppData\Local\Temp\ipykernel_11176\1408335952.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'변수명':'var','변수설명7':'var_설명'},inplace=True)


In [113]:
## merge
cor_df2 = pd.merge(cor_df,dis_df.iloc[:,[0,1]]) # dis를 중심으로 merge
cor_df2 = pd.merge(cor_df2,var_df.iloc[:,[0,1]]) # var를 중심으로 merge
out_cor = cor_df2.sort_values('dis')
out_cor

,dis,var,Cor,dis_설명,var_설명
258,DC1_PR,DK4_PT,0.500,위암 현재 유병 여부,간경변증 치료
25,DC1_PR,DC7_AG,0.693,위암 현재 유병 여부,갑상선암 진단시기
362,DC1_PR,EC_STT_2,0.524,위암 현재 유병 여부,(취업자) 종사상지위_임금근로자 상세
360,DC1_PR,BM13_3,0.461,위암 현재 유병 여부,치아손상 사유: 교통사고
132,DC1_PR,DK4_AG,0.803,위암 현재 유병 여부,간경변증 진단시기
...,...,...,...,...,...
193,DM4_PR,AC3_1_02,-0.436,골다공증 현재 유병 여부,손상2 발생시기-년도
188,DM4_PR,DC6_PT,0.471,골다공증 현재 유병 여부,폐암 치료 여부
128,DM4_PR,DK4_AG,-0.486,골다공증 현재 유병 여부,간경변증 진단시기
179,DM4_PR,DC1_AG,-0.430,골다공증 현재 유병 여부,위암 진단시기


그 외 기타 작업

In [147]:
##### 진단 시기(_AG) 빼기 작업 ###
out_cor2 = out_cor[~out_cor['var'].str.contains('_AG')] # ' _AG'를 포함하고 있는 변수를 제외한 나머지 변수 추출
len(set(out_cor2.iloc[:,1]))

183

In [122]:
# 완성된 df를 csv로 내보내기
out_cor.to_csv('cor_output.csv',encoding='cp949') ## 진단시기 변수 포함
out_cor2.to_csv('cor2_output.csv',encoding='cp949') ## 진단시기 변수 빠짐